In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torch.utils.data import Dataset, ConcatDataset

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform_grey = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.Grayscale(num_output_channels=3)])

batch_size = 6

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = transform)

trainset_grey = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = transform_grey)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
input1=[]
label=[]
for i, data in enumerate(trainset):
    inputs, labels = data
    input1.append(inputs)
    label.append(labels)

# Define your condition (e.g., labels above a certain threshold)
threshold = 4
condition_indices = [i for i, j in enumerate(label) if j > threshold]
non_condition_indices = [i for i, j in enumerate(label) if j <= threshold]

GREYSET = torch.utils.data.Subset(trainset_grey, condition_indices)

COLORSET = torch.utils.data.Subset(trainset, non_condition_indices)


In [9]:
FINAL = ConcatDataset([GREYSET, COLORSET])


trainloader = torch.utils.data.DataLoader(FINAL, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

trainloader2 = torch.utils.data.DataLoader(trainset_grey, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [10]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        # implementing other type of resnet where instead of adding original layer , adding the convoluted layer of kernel 1
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        # Inheriting from the same parent node
        super(ResNet, self).__init__()
        self.in_planes = 64
        # 1. kernel 7 by 7 has to be used then max pooling, instead used 3 by 3 to get simialr output
        # self.conv1 = nn.Conv2d(input channels,output channels , kernel_size,stride, padding, bias=False)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,stride=1, padding=1, bias=False)
        # batch normalization
        self.bn1 = nn.BatchNorm2d(64)
        # then calling the sequential layers ( Basicblock, 64, 2 , 1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        # builds 2 layers everytime as strides are 2 everytime to build resnet
        for stride in strides:
            # basicblock() 
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        # setting the dimensions by reshaping it , just cvonverting 4d into 2d
        out = self.linear(out)
        return out


def ResNet18():
    # 1. Resnet in_it runs and 
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [11]:
net = ResNet18()

In [12]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,  momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [15]:
train(2)


Epoch: 2
Loss: 2.182 | Acc: 33.333% (2/6)
Loss: 2.457 | Acc: 16.667% (2/12)
Loss: 2.447 | Acc: 11.111% (2/18)
Loss: 2.506 | Acc: 8.333% (2/24)
Loss: 2.731 | Acc: 6.667% (2/30)
Loss: 2.967 | Acc: 5.556% (2/36)
Loss: 2.933 | Acc: 9.524% (4/42)
Loss: 3.093 | Acc: 12.500% (6/48)
Loss: 3.097 | Acc: 11.111% (6/54)
Loss: 3.229 | Acc: 10.000% (6/60)
Loss: 3.218 | Acc: 10.606% (7/66)
Loss: 3.219 | Acc: 11.111% (8/72)
Loss: 3.326 | Acc: 10.256% (8/78)
Loss: 3.519 | Acc: 9.524% (8/84)
Loss: 3.595 | Acc: 8.889% (8/90)
Loss: 3.650 | Acc: 10.417% (10/96)
Loss: 3.741 | Acc: 9.804% (10/102)
Loss: 3.719 | Acc: 11.111% (12/108)
Loss: 3.665 | Acc: 11.404% (13/114)
Loss: 3.626 | Acc: 10.833% (13/120)
Loss: 3.708 | Acc: 10.317% (13/126)
Loss: 3.735 | Acc: 9.848% (13/132)
Loss: 3.705 | Acc: 9.420% (13/138)
Loss: 3.726 | Acc: 9.722% (14/144)
Loss: 3.744 | Acc: 10.000% (15/150)
Loss: 3.719 | Acc: 10.897% (17/156)
Loss: 3.797 | Acc: 10.494% (17/162)
Loss: 3.803 | Acc: 10.714% (18/168)
Loss: 3.799 | Acc: 10.34

Loss: 2.835 | Acc: 11.281% (155/1374)
Loss: 2.833 | Acc: 11.232% (155/1380)
Loss: 2.831 | Acc: 11.255% (156/1386)
Loss: 2.828 | Acc: 11.351% (158/1392)
Loss: 2.823 | Acc: 11.516% (161/1398)
Loss: 2.820 | Acc: 11.538% (162/1404)
Loss: 2.818 | Acc: 11.489% (162/1410)
Loss: 2.816 | Acc: 11.511% (163/1416)
Loss: 2.816 | Acc: 11.463% (163/1422)
Loss: 2.812 | Acc: 11.485% (164/1428)
Loss: 2.809 | Acc: 11.506% (165/1434)
Loss: 2.809 | Acc: 11.458% (165/1440)
Loss: 2.804 | Acc: 11.618% (168/1446)
Loss: 2.800 | Acc: 11.708% (170/1452)
Loss: 2.798 | Acc: 11.728% (171/1458)
Loss: 2.797 | Acc: 11.749% (172/1464)
Loss: 2.794 | Acc: 11.769% (173/1470)
Loss: 2.792 | Acc: 11.789% (174/1476)
Loss: 2.790 | Acc: 11.808% (175/1482)
Loss: 2.790 | Acc: 11.828% (176/1488)
Loss: 2.787 | Acc: 11.847% (177/1494)
Loss: 2.789 | Acc: 11.800% (177/1500)
Loss: 2.787 | Acc: 11.753% (177/1506)
Loss: 2.784 | Acc: 11.772% (178/1512)
Loss: 2.785 | Acc: 11.792% (179/1518)
Loss: 2.784 | Acc: 11.745% (179/1524)
Loss: 2.780 

Loss: 2.533 | Acc: 14.906% (398/2670)
Loss: 2.531 | Acc: 14.910% (399/2676)
Loss: 2.530 | Acc: 14.952% (401/2682)
Loss: 2.529 | Acc: 14.918% (401/2688)
Loss: 2.527 | Acc: 14.959% (403/2694)
Loss: 2.525 | Acc: 14.963% (404/2700)
Loss: 2.524 | Acc: 14.967% (405/2706)
Loss: 2.523 | Acc: 14.934% (405/2712)
Loss: 2.522 | Acc: 14.974% (407/2718)
Loss: 2.521 | Acc: 15.015% (409/2724)
Loss: 2.522 | Acc: 15.018% (410/2730)
Loss: 2.521 | Acc: 15.058% (412/2736)
Loss: 2.520 | Acc: 15.062% (413/2742)
Loss: 2.518 | Acc: 15.102% (415/2748)
Loss: 2.517 | Acc: 15.105% (416/2754)
Loss: 2.515 | Acc: 15.145% (418/2760)
Loss: 2.515 | Acc: 15.148% (419/2766)
Loss: 2.514 | Acc: 15.224% (422/2772)
Loss: 2.512 | Acc: 15.263% (424/2778)
Loss: 2.512 | Acc: 15.266% (425/2784)
Loss: 2.512 | Acc: 15.269% (426/2790)
Loss: 2.512 | Acc: 15.272% (427/2796)
Loss: 2.512 | Acc: 15.275% (428/2802)
Loss: 2.513 | Acc: 15.313% (430/2808)
Loss: 2.512 | Acc: 15.316% (431/2814)
Loss: 2.511 | Acc: 15.284% (431/2820)
Loss: 2.511 

Loss: 2.343 | Acc: 18.936% (751/3966)
Loss: 2.343 | Acc: 18.983% (754/3972)
Loss: 2.342 | Acc: 19.005% (756/3978)
Loss: 2.341 | Acc: 19.001% (757/3984)
Loss: 2.340 | Acc: 19.023% (759/3990)
Loss: 2.340 | Acc: 19.019% (760/3996)
Loss: 2.339 | Acc: 19.015% (761/4002)
Loss: 2.338 | Acc: 19.012% (762/4008)
Loss: 2.338 | Acc: 19.008% (763/4014)
Loss: 2.337 | Acc: 19.005% (764/4020)
Loss: 2.336 | Acc: 19.051% (767/4026)
Loss: 2.335 | Acc: 19.072% (769/4032)
Loss: 2.335 | Acc: 19.044% (769/4038)
Loss: 2.334 | Acc: 19.041% (770/4044)
Loss: 2.334 | Acc: 19.037% (771/4050)
Loss: 2.334 | Acc: 19.034% (772/4056)
Loss: 2.332 | Acc: 19.104% (776/4062)
Loss: 2.331 | Acc: 19.125% (778/4068)
Loss: 2.330 | Acc: 19.146% (780/4074)
Loss: 2.329 | Acc: 19.216% (784/4080)
Loss: 2.329 | Acc: 19.212% (785/4086)
Loss: 2.328 | Acc: 19.233% (787/4092)
Loss: 2.327 | Acc: 19.229% (788/4098)
Loss: 2.326 | Acc: 19.225% (789/4104)
Loss: 2.325 | Acc: 19.221% (790/4110)
Loss: 2.323 | Acc: 19.315% (795/4116)
Loss: 2.322 

Loss: 2.220 | Acc: 21.943% (1152/5250)
Loss: 2.220 | Acc: 21.975% (1155/5256)
Loss: 2.220 | Acc: 21.969% (1156/5262)
Loss: 2.219 | Acc: 22.001% (1159/5268)
Loss: 2.219 | Acc: 21.995% (1160/5274)
Loss: 2.219 | Acc: 22.008% (1162/5280)
Loss: 2.218 | Acc: 22.020% (1164/5286)
Loss: 2.218 | Acc: 22.033% (1166/5292)
Loss: 2.218 | Acc: 22.065% (1169/5298)
Loss: 2.217 | Acc: 22.078% (1171/5304)
Loss: 2.217 | Acc: 22.072% (1172/5310)
Loss: 2.216 | Acc: 22.065% (1173/5316)
Loss: 2.216 | Acc: 22.097% (1176/5322)
Loss: 2.215 | Acc: 22.128% (1179/5328)
Loss: 2.216 | Acc: 22.122% (1180/5334)
Loss: 2.216 | Acc: 22.116% (1181/5340)
Loss: 2.215 | Acc: 22.147% (1184/5346)
Loss: 2.214 | Acc: 22.123% (1184/5352)
Loss: 2.215 | Acc: 22.098% (1184/5358)
Loss: 2.214 | Acc: 22.148% (1188/5364)
Loss: 2.213 | Acc: 22.216% (1193/5370)
Loss: 2.212 | Acc: 22.228% (1195/5376)
Loss: 2.211 | Acc: 22.278% (1199/5382)
Loss: 2.211 | Acc: 22.272% (1200/5388)
Loss: 2.211 | Acc: 22.247% (1200/5394)
Loss: 2.211 | Acc: 22.259

...

Loss: 2.399 | Acc: 57.617% (54511/94610)
Loss: 2.398 | Acc: 57.618% (54516/94616)
Loss: 2.398 | Acc: 57.620% (54521/94622)
Loss: 2.398 | Acc: 57.621% (54526/94628)
Loss: 2.398 | Acc: 57.623% (54531/94634)
Loss: 2.397 | Acc: 57.626% (54537/94640)
Loss: 2.397 | Acc: 57.627% (54542/94646)
Loss: 2.397 | Acc: 57.628% (54546/94652)
Loss: 2.397 | Acc: 57.627% (54549/94658)
Loss: 2.397 | Acc: 57.628% (54553/94664)
Loss: 2.396 | Acc: 57.629% (54557/94670)
Loss: 2.396 | Acc: 57.631% (54563/94676)
Loss: 2.396 | Acc: 57.630% (54565/94682)
Loss: 2.396 | Acc: 57.632% (54571/94688)
Loss: 2.396 | Acc: 57.633% (54575/94694)
Loss: 2.395 | Acc: 57.635% (54580/94700)
Loss: 2.395 | Acc: 57.637% (54586/94706)
Loss: 2.395 | Acc: 57.640% (54592/94712)
Loss: 2.395 | Acc: 57.642% (54597/94718)
Loss: 2.394 | Acc: 57.643% (54602/94724)
Loss: 2.394 | Acc: 57.646% (54608/94730)
Loss: 2.394 | Acc: 57.648% (54613/94736)
Loss: 2.394 | Acc: 57.648% (54617/94742)
Loss: 2.393 | Acc: 57.650% (54622/94748)
Loss: 2.393 | Ac

Loss: 2.356 | Acc: 57.798% (55376/95810)
Loss: 2.355 | Acc: 57.799% (55381/95816)
Loss: 2.355 | Acc: 57.802% (55387/95822)
Loss: 2.355 | Acc: 57.804% (55392/95828)
Loss: 2.355 | Acc: 57.804% (55396/95834)
Loss: 2.355 | Acc: 57.803% (55398/95840)
Loss: 2.354 | Acc: 57.802% (55401/95846)
Loss: 2.354 | Acc: 57.804% (55406/95852)
Loss: 2.354 | Acc: 57.806% (55412/95858)
Loss: 2.354 | Acc: 57.806% (55415/95864)
Loss: 2.354 | Acc: 57.807% (55420/95870)
Loss: 2.353 | Acc: 57.810% (55426/95876)
Loss: 2.353 | Acc: 57.813% (55432/95882)
Loss: 2.353 | Acc: 57.814% (55437/95888)
Loss: 2.353 | Acc: 57.814% (55440/95894)
Loss: 2.352 | Acc: 57.814% (55444/95900)
Loss: 2.352 | Acc: 57.817% (55450/95906)
Loss: 2.352 | Acc: 57.818% (55454/95912)
Loss: 2.352 | Acc: 57.819% (55459/95918)
Loss: 2.352 | Acc: 57.822% (55465/95924)
Loss: 2.351 | Acc: 57.822% (55469/95930)
Loss: 2.351 | Acc: 57.824% (55474/95936)
Loss: 2.351 | Acc: 57.826% (55479/95942)
Loss: 2.351 | Acc: 57.826% (55483/95948)
Loss: 2.350 | Ac

Loss: 2.313 | Acc: 58.005% (56271/97010)
Loss: 2.313 | Acc: 58.005% (56274/97016)
Loss: 2.313 | Acc: 58.006% (56279/97022)
Loss: 2.313 | Acc: 58.007% (56283/97028)
Loss: 2.313 | Acc: 58.008% (56287/97034)
Loss: 2.312 | Acc: 58.008% (56291/97040)
Loss: 2.312 | Acc: 58.011% (56297/97046)
Loss: 2.312 | Acc: 58.011% (56301/97052)
Loss: 2.312 | Acc: 58.013% (56306/97058)
Loss: 2.311 | Acc: 58.014% (56311/97064)
Loss: 2.311 | Acc: 58.015% (56315/97070)
Loss: 2.311 | Acc: 58.016% (56320/97076)
Loss: 2.311 | Acc: 58.016% (56323/97082)
Loss: 2.311 | Acc: 58.017% (56328/97088)
Loss: 2.310 | Acc: 58.020% (56334/97094)
Loss: 2.310 | Acc: 58.022% (56339/97100)
Loss: 2.310 | Acc: 58.022% (56343/97106)
Loss: 2.310 | Acc: 58.023% (56347/97112)
Loss: 2.310 | Acc: 58.023% (56351/97118)
Loss: 2.309 | Acc: 58.025% (56356/97124)
Loss: 2.309 | Acc: 58.025% (56360/97130)
Loss: 2.309 | Acc: 58.027% (56365/97136)
Loss: 2.309 | Acc: 58.027% (56369/97142)
Loss: 2.309 | Acc: 58.028% (56373/97148)
Loss: 2.308 | Ac

Loss: 2.274 | Acc: 58.187% (57145/98210)
Loss: 2.274 | Acc: 58.187% (57149/98216)
Loss: 2.274 | Acc: 58.187% (57152/98222)
Loss: 2.274 | Acc: 58.187% (57156/98228)
Loss: 2.274 | Acc: 58.187% (57159/98234)
Loss: 2.273 | Acc: 58.189% (57165/98240)
Loss: 2.273 | Acc: 58.192% (57171/98246)
Loss: 2.273 | Acc: 58.189% (57172/98252)
Loss: 2.273 | Acc: 58.192% (57178/98258)
Loss: 2.273 | Acc: 58.193% (57183/98264)
Loss: 2.272 | Acc: 58.193% (57186/98270)
Loss: 2.272 | Acc: 58.194% (57191/98276)
Loss: 2.272 | Acc: 58.195% (57195/98282)
Loss: 2.272 | Acc: 58.196% (57200/98288)
Loss: 2.272 | Acc: 58.196% (57203/98294)
Loss: 2.272 | Acc: 58.196% (57207/98300)
Loss: 2.271 | Acc: 58.197% (57211/98306)
Loss: 2.271 | Acc: 58.198% (57216/98312)
Loss: 2.271 | Acc: 58.199% (57220/98318)
Loss: 2.271 | Acc: 58.201% (57226/98324)
Loss: 2.271 | Acc: 58.202% (57230/98330)
Loss: 2.270 | Acc: 58.202% (57234/98336)
Loss: 2.270 | Acc: 58.203% (57238/98342)
Loss: 2.270 | Acc: 58.204% (57242/98348)
Loss: 2.270 | Ac

Loss: 2.236 | Acc: 58.383% (58039/99410)
Loss: 2.236 | Acc: 58.385% (58044/99416)
Loss: 2.236 | Acc: 58.385% (58048/99422)
Loss: 2.236 | Acc: 58.385% (58051/99428)
Loss: 2.236 | Acc: 58.386% (58056/99434)
Loss: 2.235 | Acc: 58.387% (58060/99440)
Loss: 2.235 | Acc: 58.387% (58064/99446)
Loss: 2.235 | Acc: 58.386% (58066/99452)
Loss: 2.235 | Acc: 58.385% (58069/99458)
Loss: 2.235 | Acc: 58.386% (58073/99464)
Loss: 2.234 | Acc: 58.388% (58079/99470)
Loss: 2.234 | Acc: 58.388% (58082/99476)
Loss: 2.234 | Acc: 58.389% (58087/99482)
Loss: 2.234 | Acc: 58.389% (58090/99488)
Loss: 2.234 | Acc: 58.390% (58095/99494)
Loss: 2.234 | Acc: 58.391% (58099/99500)
Loss: 2.234 | Acc: 58.392% (58104/99506)
Loss: 2.233 | Acc: 58.392% (58107/99512)
Loss: 2.233 | Acc: 58.392% (58111/99518)
Loss: 2.233 | Acc: 58.393% (58115/99524)
Loss: 2.233 | Acc: 58.392% (58118/99530)
Loss: 2.233 | Acc: 58.394% (58123/99536)
Loss: 2.232 | Acc: 58.395% (58128/99542)
Loss: 2.232 | Acc: 58.398% (58134/99548)
Loss: 2.232 | Ac

In [16]:
total = 0 
correct =0


testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

with torch.no_grad():
    
    for data in testloader:
    
        inputs, labels=data

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Files already downloaded and verified
Accuracy of the network on the 10000 test images: 37 %


In [17]:
net = ResNet18()

In [18]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,  momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train2(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader2):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


In [19]:
train2(2)


Epoch: 2
Loss: 2.513 | Acc: 0.000% (0/6)
Loss: 2.561 | Acc: 0.000% (0/12)
Loss: 2.588 | Acc: 0.000% (0/18)
Loss: 2.571 | Acc: 8.333% (2/24)
Loss: 2.529 | Acc: 10.000% (3/30)
Loss: 2.765 | Acc: 8.333% (3/36)
Loss: 3.006 | Acc: 7.143% (3/42)
Loss: 2.913 | Acc: 10.417% (5/48)
Loss: 2.921 | Acc: 9.259% (5/54)
Loss: 3.031 | Acc: 10.000% (6/60)
Loss: 3.061 | Acc: 10.606% (7/66)
Loss: 3.103 | Acc: 11.111% (8/72)
Loss: 3.172 | Acc: 11.538% (9/78)
Loss: 3.215 | Acc: 11.905% (10/84)
Loss: 3.263 | Acc: 11.111% (10/90)
Loss: 3.445 | Acc: 10.417% (10/96)
Loss: 3.527 | Acc: 10.784% (11/102)
Loss: 3.439 | Acc: 12.963% (14/108)
Loss: 3.567 | Acc: 12.281% (14/114)
Loss: 3.551 | Acc: 12.500% (15/120)
Loss: 3.536 | Acc: 12.698% (16/126)
Loss: 3.579 | Acc: 13.636% (18/132)
Loss: 3.620 | Acc: 13.043% (18/138)
Loss: 3.597 | Acc: 13.889% (20/144)
Loss: 3.567 | Acc: 14.667% (22/150)
Loss: 3.591 | Acc: 16.026% (25/156)
Loss: 3.669 | Acc: 16.049% (26/162)
Loss: 3.719 | Acc: 15.476% (26/168)
Loss: 3.663 | Acc: 

Loss: 2.793 | Acc: 14.741% (199/1350)
Loss: 2.792 | Acc: 14.676% (199/1356)
Loss: 2.790 | Acc: 14.611% (199/1362)
Loss: 2.788 | Acc: 14.620% (200/1368)
Loss: 2.785 | Acc: 14.629% (201/1374)
Loss: 2.783 | Acc: 14.565% (201/1380)
Loss: 2.782 | Acc: 14.502% (201/1386)
Loss: 2.781 | Acc: 14.583% (203/1392)
Loss: 2.779 | Acc: 14.521% (203/1398)
Loss: 2.777 | Acc: 14.530% (204/1404)
Loss: 2.776 | Acc: 14.468% (204/1410)
Loss: 2.775 | Acc: 14.407% (204/1416)
Loss: 2.772 | Acc: 14.487% (206/1422)
Loss: 2.770 | Acc: 14.426% (206/1428)
Loss: 2.769 | Acc: 14.505% (208/1434)
Loss: 2.767 | Acc: 14.444% (208/1440)
Loss: 2.766 | Acc: 14.523% (210/1446)
Loss: 2.766 | Acc: 14.463% (210/1452)
Loss: 2.764 | Acc: 14.403% (210/1458)
Loss: 2.762 | Acc: 14.413% (211/1464)
Loss: 2.762 | Acc: 14.422% (212/1470)
Loss: 2.759 | Acc: 14.431% (213/1476)
Loss: 2.759 | Acc: 14.440% (214/1482)
Loss: 2.757 | Acc: 14.449% (215/1488)
Loss: 2.757 | Acc: 14.391% (215/1494)
Loss: 2.754 | Acc: 14.400% (216/1500)
Loss: 2.753 

Loss: 2.540 | Acc: 16.251% (430/2646)
Loss: 2.539 | Acc: 16.252% (431/2652)
Loss: 2.538 | Acc: 16.290% (433/2658)
Loss: 2.537 | Acc: 16.254% (433/2664)
Loss: 2.537 | Acc: 16.255% (434/2670)
Loss: 2.536 | Acc: 16.256% (435/2676)
Loss: 2.534 | Acc: 16.294% (437/2682)
Loss: 2.534 | Acc: 16.295% (438/2688)
Loss: 2.534 | Acc: 16.295% (439/2694)
Loss: 2.532 | Acc: 16.333% (441/2700)
Loss: 2.531 | Acc: 16.334% (442/2706)
Loss: 2.531 | Acc: 16.298% (442/2712)
Loss: 2.531 | Acc: 16.262% (442/2718)
Loss: 2.531 | Acc: 16.226% (442/2724)
Loss: 2.531 | Acc: 16.190% (442/2730)
Loss: 2.531 | Acc: 16.192% (443/2736)
Loss: 2.530 | Acc: 16.229% (445/2742)
Loss: 2.528 | Acc: 16.303% (448/2748)
Loss: 2.527 | Acc: 16.304% (449/2754)
Loss: 2.526 | Acc: 16.341% (451/2760)
Loss: 2.526 | Acc: 16.341% (452/2766)
Loss: 2.526 | Acc: 16.342% (453/2772)
Loss: 2.526 | Acc: 16.343% (454/2778)
Loss: 2.525 | Acc: 16.307% (454/2784)
Loss: 2.522 | Acc: 16.380% (457/2790)
Loss: 2.521 | Acc: 16.416% (459/2796)
Loss: 2.521 

Loss: 2.413 | Acc: 17.428% (687/3942)
Loss: 2.413 | Acc: 17.452% (689/3948)
Loss: 2.413 | Acc: 17.451% (690/3954)
Loss: 2.412 | Acc: 17.475% (692/3960)
Loss: 2.412 | Acc: 17.474% (693/3966)
Loss: 2.411 | Acc: 17.447% (693/3972)
Loss: 2.410 | Acc: 17.471% (695/3978)
Loss: 2.410 | Acc: 17.470% (696/3984)
Loss: 2.409 | Acc: 17.469% (697/3990)
Loss: 2.410 | Acc: 17.467% (698/3996)
Loss: 2.409 | Acc: 17.491% (700/4002)
Loss: 2.408 | Acc: 17.540% (703/4008)
Loss: 2.409 | Acc: 17.514% (703/4014)
Loss: 2.409 | Acc: 17.488% (703/4020)
Loss: 2.408 | Acc: 17.511% (705/4026)
Loss: 2.407 | Acc: 17.510% (706/4032)
Loss: 2.408 | Acc: 17.484% (706/4038)
Loss: 2.407 | Acc: 17.483% (707/4044)
Loss: 2.407 | Acc: 17.457% (707/4050)
Loss: 2.407 | Acc: 17.431% (707/4056)
Loss: 2.406 | Acc: 17.454% (709/4062)
Loss: 2.405 | Acc: 17.502% (712/4068)
Loss: 2.405 | Acc: 17.501% (713/4074)
Loss: 2.404 | Acc: 17.500% (714/4080)
Loss: 2.404 | Acc: 17.499% (715/4086)
Loss: 2.403 | Acc: 17.571% (719/4092)
Loss: 2.403 

Loss: 2.340 | Acc: 18.461% (967/5238)
Loss: 2.340 | Acc: 18.459% (968/5244)
Loss: 2.340 | Acc: 18.476% (970/5250)
Loss: 2.340 | Acc: 18.474% (971/5256)
Loss: 2.339 | Acc: 18.491% (973/5262)
Loss: 2.339 | Acc: 18.508% (975/5268)
Loss: 2.339 | Acc: 18.506% (976/5274)
Loss: 2.338 | Acc: 18.504% (977/5280)
Loss: 2.338 | Acc: 18.502% (978/5286)
Loss: 2.338 | Acc: 18.500% (979/5292)
Loss: 2.337 | Acc: 18.498% (980/5298)
Loss: 2.337 | Acc: 18.495% (981/5304)
Loss: 2.337 | Acc: 18.475% (981/5310)
Loss: 2.337 | Acc: 18.473% (982/5316)
Loss: 2.336 | Acc: 18.452% (982/5322)
Loss: 2.336 | Acc: 18.487% (985/5328)
Loss: 2.335 | Acc: 18.504% (987/5334)
Loss: 2.335 | Acc: 18.521% (989/5340)
Loss: 2.334 | Acc: 18.575% (993/5346)
Loss: 2.334 | Acc: 18.572% (994/5352)
Loss: 2.334 | Acc: 18.570% (995/5358)
Loss: 2.333 | Acc: 18.568% (996/5364)
Loss: 2.333 | Acc: 18.566% (997/5370)
Loss: 2.333 | Acc: 18.545% (997/5376)
Loss: 2.333 | Acc: 18.580% (1000/5382)
Loss: 2.332 | Acc: 18.597% (1002/5388)
Loss: 2.33

...

Loss: 1.726 | Acc: 37.497% (16687/44502)
Loss: 1.726 | Acc: 37.499% (16690/44508)
Loss: 1.726 | Acc: 37.501% (16693/44514)
Loss: 1.726 | Acc: 37.504% (16697/44520)
Loss: 1.726 | Acc: 37.511% (16702/44526)
Loss: 1.726 | Acc: 37.512% (16705/44532)
Loss: 1.726 | Acc: 37.514% (16708/44538)
Loss: 1.726 | Acc: 37.513% (16710/44544)
Loss: 1.726 | Acc: 37.517% (16714/44550)
Loss: 1.726 | Acc: 37.517% (16716/44556)
Loss: 1.725 | Acc: 37.523% (16721/44562)
Loss: 1.725 | Acc: 37.527% (16725/44568)
Loss: 1.725 | Acc: 37.526% (16727/44574)
Loss: 1.725 | Acc: 37.533% (16732/44580)
Loss: 1.725 | Acc: 37.534% (16735/44586)
Loss: 1.725 | Acc: 37.538% (16739/44592)
Loss: 1.725 | Acc: 37.540% (16742/44598)
Loss: 1.725 | Acc: 37.544% (16746/44604)
Loss: 1.725 | Acc: 37.550% (16751/44610)
Loss: 1.725 | Acc: 37.554% (16755/44616)
Loss: 1.725 | Acc: 37.558% (16759/44622)
Loss: 1.725 | Acc: 37.559% (16762/44628)
Loss: 1.725 | Acc: 37.557% (16763/44634)
Loss: 1.725 | Acc: 37.558% (16766/44640)
Loss: 1.725 | Ac

Loss: 1.714 | Acc: 37.946% (17342/45702)
Loss: 1.714 | Acc: 37.947% (17345/45708)
Loss: 1.714 | Acc: 37.949% (17348/45714)
Loss: 1.714 | Acc: 37.948% (17350/45720)
Loss: 1.714 | Acc: 37.948% (17352/45726)
Loss: 1.714 | Acc: 37.947% (17354/45732)
Loss: 1.714 | Acc: 37.951% (17358/45738)
Loss: 1.714 | Acc: 37.953% (17361/45744)
Loss: 1.714 | Acc: 37.961% (17367/45750)
Loss: 1.714 | Acc: 37.962% (17370/45756)
Loss: 1.714 | Acc: 37.964% (17373/45762)
Loss: 1.714 | Acc: 37.965% (17376/45768)
Loss: 1.714 | Acc: 37.971% (17381/45774)
Loss: 1.714 | Acc: 37.971% (17383/45780)
Loss: 1.714 | Acc: 37.970% (17385/45786)
Loss: 1.713 | Acc: 37.972% (17388/45792)
Loss: 1.713 | Acc: 37.973% (17391/45798)
Loss: 1.713 | Acc: 37.973% (17393/45804)
Loss: 1.713 | Acc: 37.974% (17396/45810)
Loss: 1.713 | Acc: 37.978% (17400/45816)
Loss: 1.713 | Acc: 37.980% (17403/45822)
Loss: 1.713 | Acc: 37.981% (17406/45828)
Loss: 1.713 | Acc: 37.983% (17409/45834)
Loss: 1.713 | Acc: 37.984% (17412/45840)
Loss: 1.713 | Ac

Loss: 1.702 | Acc: 38.357% (17990/46902)
Loss: 1.702 | Acc: 38.360% (17994/46908)
Loss: 1.702 | Acc: 38.364% (17998/46914)
Loss: 1.702 | Acc: 38.370% (18003/46920)
Loss: 1.702 | Acc: 38.373% (18007/46926)
Loss: 1.702 | Acc: 38.373% (18009/46932)
Loss: 1.702 | Acc: 38.380% (18015/46938)
Loss: 1.702 | Acc: 38.382% (18018/46944)
Loss: 1.702 | Acc: 38.388% (18023/46950)
Loss: 1.702 | Acc: 38.387% (18025/46956)
Loss: 1.701 | Acc: 38.388% (18028/46962)
Loss: 1.701 | Acc: 38.390% (18031/46968)
Loss: 1.701 | Acc: 38.394% (18035/46974)
Loss: 1.701 | Acc: 38.401% (18041/46980)
Loss: 1.701 | Acc: 38.399% (18042/46986)
Loss: 1.701 | Acc: 38.404% (18047/46992)
Loss: 1.701 | Acc: 38.404% (18049/46998)
Loss: 1.701 | Acc: 38.407% (18053/47004)
Loss: 1.701 | Acc: 38.413% (18058/47010)
Loss: 1.701 | Acc: 38.415% (18061/47016)
Loss: 1.701 | Acc: 38.416% (18064/47022)
Loss: 1.701 | Acc: 38.418% (18067/47028)
Loss: 1.701 | Acc: 38.419% (18070/47034)
Loss: 1.701 | Acc: 38.423% (18074/47040)
Loss: 1.701 | Ac

Loss: 1.692 | Acc: 38.793% (18660/48102)
Loss: 1.692 | Acc: 38.798% (18665/48108)
Loss: 1.692 | Acc: 38.797% (18667/48114)
Loss: 1.692 | Acc: 38.797% (18669/48120)
Loss: 1.692 | Acc: 38.798% (18672/48126)
Loss: 1.692 | Acc: 38.806% (18678/48132)
Loss: 1.692 | Acc: 38.807% (18681/48138)
Loss: 1.692 | Acc: 38.811% (18685/48144)
Loss: 1.691 | Acc: 38.816% (18690/48150)
Loss: 1.691 | Acc: 38.820% (18694/48156)
Loss: 1.691 | Acc: 38.819% (18696/48162)
Loss: 1.691 | Acc: 38.822% (18700/48168)
Loss: 1.691 | Acc: 38.826% (18704/48174)
Loss: 1.691 | Acc: 38.831% (18709/48180)
Loss: 1.691 | Acc: 38.835% (18713/48186)
Loss: 1.691 | Acc: 38.836% (18716/48192)
Loss: 1.691 | Acc: 38.840% (18720/48198)
Loss: 1.691 | Acc: 38.843% (18724/48204)
Loss: 1.691 | Acc: 38.849% (18729/48210)
Loss: 1.691 | Acc: 38.846% (18730/48216)
Loss: 1.691 | Acc: 38.847% (18733/48222)
Loss: 1.691 | Acc: 38.849% (18736/48228)
Loss: 1.691 | Acc: 38.850% (18739/48234)
Loss: 1.691 | Acc: 38.854% (18743/48240)
Loss: 1.690 | Ac

Loss: 1.680 | Acc: 39.250% (19351/49302)
Loss: 1.680 | Acc: 39.251% (19354/49308)
Loss: 1.680 | Acc: 39.257% (19359/49314)
Loss: 1.679 | Acc: 39.262% (19364/49320)
Loss: 1.679 | Acc: 39.265% (19368/49326)
Loss: 1.679 | Acc: 39.265% (19370/49332)
Loss: 1.679 | Acc: 39.268% (19374/49338)
Loss: 1.679 | Acc: 39.269% (19377/49344)
Loss: 1.679 | Acc: 39.268% (19379/49350)
Loss: 1.679 | Acc: 39.268% (19381/49356)
Loss: 1.679 | Acc: 39.269% (19384/49362)
Loss: 1.679 | Acc: 39.272% (19388/49368)
Loss: 1.679 | Acc: 39.276% (19392/49374)
Loss: 1.679 | Acc: 39.277% (19395/49380)
Loss: 1.679 | Acc: 39.280% (19399/49386)
Loss: 1.679 | Acc: 39.284% (19403/49392)
Loss: 1.679 | Acc: 39.283% (19405/49398)
Loss: 1.679 | Acc: 39.282% (19407/49404)
Loss: 1.679 | Acc: 39.288% (19412/49410)
Loss: 1.679 | Acc: 39.285% (19413/49416)
Loss: 1.678 | Acc: 39.290% (19418/49422)
Loss: 1.678 | Acc: 39.289% (19420/49428)
Loss: 1.678 | Acc: 39.295% (19425/49434)
Loss: 1.678 | Acc: 39.296% (19428/49440)
Loss: 1.678 | Ac

In [20]:
total = 0 
correct =0

with torch.no_grad():
    
    for data in testloader:
    
        inputs, labels=data

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 56 %


In [2]:

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

NameError: name 'torchvision' is not defined